In [22]:
# Import dependencies
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc
from sqlalchemy.sql import label
from sqlalchemy import and_

import matplotlib.pyplot as plt

In [23]:
#create engine
engine = create_engine("sqlite:///hawaii.sqlite", echo = True)

In [24]:
#prepare and reflect the database
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

2018-04-07 03:53:42,387 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-07 03:53:42,389 INFO sqlalchemy.engine.base.Engine ()
2018-04-07 03:53:42,392 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-07 03:53:42,394 INFO sqlalchemy.engine.base.Engine ()
2018-04-07 03:53:42,397 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-04-07 03:53:42,399 INFO sqlalchemy.engine.base.Engine ()
2018-04-07 03:53:42,402 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurement")
2018-04-07 03:53:42,404 INFO sqlalchemy.engine.base.Engine ()
2018-04-07 03:53:42,408 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'measurement' AND type = 'table'
2018-04-07 03:53:42,410 INFO sqlalchemy.engine.base.Engine ()
2018-04-07 03:53:42,414 INFO sqlalchemy.

['measurement', 'station']

In [25]:
#Save each table as their own reference classes
Measurement = Base.classes.measurement
Station = Base.classes.station

In [26]:
# Create our session (link) from Python to the DB
session = Session(bind = engine)

In [27]:
#Use the session to query Measurement table and display the first 5 precipitation values
session.query(Measurement.prcp).limit(5).all()

2018-04-07 03:53:42,510 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-07 03:53:42,515 INFO sqlalchemy.engine.base.Engine SELECT measurement.prcp AS measurement_prcp 
FROM measurement
 LIMIT ? OFFSET ?
2018-04-07 03:53:42,518 INFO sqlalchemy.engine.base.Engine (5, 0)


[]

In [28]:
# Getting the table names for each table
inspector = inspect(engine)
inspector.get_table_names()

2018-04-07 03:53:42,541 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-04-07 03:53:42,553 INFO sqlalchemy.engine.base.Engine ()


['measurement', 'station']

In [29]:
# Get a list of column names and types - measurement
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

2018-04-07 03:53:42,578 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurement")
2018-04-07 03:53:42,594 INFO sqlalchemy.engine.base.Engine ()
id INTEGER
station VARCHAR(255)
date VARCHAR
prcp FLOAT
tobs FLOAT


In [30]:
# Get a list of column names and types - station
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

2018-04-07 03:53:42,658 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("station")
2018-04-07 03:53:42,670 INFO sqlalchemy.engine.base.Engine ()
id INTEGER
station VARCHAR(255)
name VARCHAR(255)
latitude FLOAT
longitude FLOAT
elevation FLOAT
location FLOAT


In [31]:
engine.execute('select * from station').fetchall()

2018-04-07 03:53:42,692 INFO sqlalchemy.engine.base.Engine select * from station
2018-04-07 03:53:42,697 INFO sqlalchemy.engine.base.Engine ()


[]

In [32]:
engine.execute('SELECT * FROM measurement LIMIT 5').fetchall()

2018-04-07 03:53:42,712 INFO sqlalchemy.engine.base.Engine SELECT * FROM measurement LIMIT 5
2018-04-07 03:53:42,715 INFO sqlalchemy.engine.base.Engine ()


[]

## Precipitation Analysis

In [33]:
#query the database for a years worth of precipitation data. Focus on most recent.
precipitation_data = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= "2016-08-23").\
    filter(Measurement.date <= "2017-08-23").all()

2018-04-07 03:53:42,738 INFO sqlalchemy.engine.base.Engine SELECT measurement.date AS measurement_date, measurement.prcp AS measurement_prcp 
FROM measurement 
WHERE measurement.date >= ? AND measurement.date <= ?
2018-04-07 03:53:42,741 INFO sqlalchemy.engine.base.Engine ('2016-08-23', '2017-08-23')


In [34]:
#set index equal to date
pre_df = pd.DataFrame(precipitation_data)
pre_df.head()

""


In [35]:
pre_df['date'] = pd.to_datetime(pre_df['date'])

KeyError: 'date'

In [ ]:
prec_df = pre_df.set_index('date')
prec_df.head()

In [ ]:
#plot the data
prec_plot = prec_df.plot(figsize = (16, 9), sort_columns=True, rot=45, use_index=True, legend=True, grid=True, color='b')
plt.ylabel('Precipitation', weight='bold')
plt.xlabel('Date', weight='bold')
plt.title("Precipitation in Hawaii from 8-23-2016 to 8-23-2017")
plt.show()

In [ ]:
#show short statistical analysis for data
prec_df.describe()

## Station Analysis

In [ ]:
## Start Station Analysis
station_num = session.query(Station.id).count()
station_num

In [ ]:
#Design a query to find the most active stations.
#List the stations and observation counts in descending order
#Which station has the highest number of observations?
most_active_station = session.query(Measurements.station, Stations.name, func.count(Measurements.tobs)).\
filter(Measurements.station == Station.station).group_by(Measurement.station).order_by(func.count(Measurement.tobs).desc()).all()

In [ ]:
#Waihee 837.5 has the highest number of observations
most_active_station

In [ ]:
waihee_tobs = session.query(Measurement.tobs).\
filter(Measurement.station == "USC00519281", Measurement.station == Station.station, Measurement.date >="2016-08-23", Measurement.date <="2017-08-23").\
all()

In [ ]:
len(waihee_tobs)

In [ ]:
print(waihee_tobs[0][0])

## Temperature Analysis

In [ ]:
temperatures = [temp[0] for temp in waihee_tobs]

In [ ]:
plt.hist(temperatures, bins=12, color='b')
plt.title("Temperature Observation for Waihee Station (2016-08-23 to 2017)")
plt.ylabel('Frequency', weight='bold')
plt.xlabel('Temperature', weight='bold')
labels = ['tobs']
plt.legend(labels)

In [ ]:
def calc_temps(start, end):
    query = session.query(Measurement.tobs).filter(Measurement.date>=start, Measurement.date<=end).all()
    temperatures = [temp[0] for temp in query]
    avg_temp = np.mean(temperatures)
    lowest_temp = min(temperatures)
    highest_temp = max(temperatures)
    

    plt.figure(figsize=(3,5))
    plt.bar(1, avg_temp, yerr=(highest_temp - lowest_temp), tick_label='', color='salmon')
    plt.ylabel("Temperature", weight="bold")
    plt.title("Average Trip Temperature", weight="bold")
    plt.show()

In [ ]:
columns = inspector.get_columns('Measurement')
for column in columns:
    print(column['name'])

In [ ]:
calc_temps(start="2017-08-07", end="2017-08-23")